# SNNomics demo 

![Overview](imgs/overview.png)

## Import modules

In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from pathlib import Path
from SNNomics.network import SNN

## Load data

In [ ]:
query_sample = np.load("data/query_sample.npy")
database = np.load("data/test_database.npz")
database_expression = database["expression"]
database_ids = database['gsms']
labels = pd.read_csv('data/tissue_labels.csv', index_col=0)
onto_map = pd.read_csv('data/onto_map.csv')

## Compute similarities between the query and all other samples

In [ ]:
num_genes = len(query_sample)   # Number of genes for input dimension
num_samples = len(database_ids) # Number of samples to iterate through
model = SNN(num_genes)  # Initialize model

query = torch.from_numpy(query_sample)

results = {'id': [], 'similarity': []}
for sample in range(num_samples):
    z = torch.from_numpy(database_expression[sample, :])
    similarity = model(query, z)
    
    results['id'].append(database_ids[sample])
    results['similarity'].append(similarity.item())
    
results_df = pd.DataFrame.from_dict(results)

## Map tissue labels to samples

In [ ]:
def assign_labels(gsm: str, labels_df: pd.DataFrame, onto_map: pd.DataFrame) -> str:
    terms = labels_df.columns[labels_df.loc[gsm] == 1].tolist()
    term_names = []
    for term in terms:
        term_names.append(onto_map.loc[onto_map['id'] == term, 'name'].item())
    
    return ','.join(term_names)

results_df['tissues'] = results_df['id'].apply(assign_labels, labels_df=labels, onto_map=onto_map)


print(f"Query tissue: heart")
print(results_df.head(20))